<div class='alert-block alert-info'>
    <br>
    <h1 align="center"> <b> Lab session 1 - Part I:</b> Intelligent agents </h1> 
    <h3 align="center">Artificial Intelligence Algorithms </h3>
    <h5 align="center">MESIIN476023</a></h5>
    <br>
</div>

This notebook serves as supporting material for the topics covered by the first lecture of the AI algoritms course. 

#### CONTENTS

* **Part I: Intelligent Agents**
    * Overview
    * Agent
    * Environment
    * Example: Blind dog agent 
        - Simple Agent and Environment
        - Agents in a 2-D Environment
    * Exercise: Vacuum cleaner Agent </br>



<h1 align="left"> <font color='darkblue'> Part I: Intelligent Agents </font></a></h1>

This part uses implementations from *"agents.py"*. Let's start by importing everything from agents module.

In [57]:
from agents import Agent, Environment, Thing, GraphicEnvironment, Direction, Bump
from Complementary_notebook import psource




<h2 align="left">  I.1. OVERVIEW</h2>

An agent is anything that can perceive its <b>environment</b> through sensors, and act upon that environment through actuators based on its <b>agent program</b>. This can be a dog, a robot, or even you. As long as you can perceive the environment and act on it, you are an agent. This notebook will explain how to implement a simple agent, create an environment, and implement a program that helps the agent act on the environment based on its percepts.

<h2 align="left"> I.2. AGENT</h2>

Let us now see how we define an agent. Run the next cell to see how `Agent` is defined in agents module.

In [58]:
psource(Agent)

The `Agent` has two methods.
* `__init__(self, program=None)`: The constructor defines various attributes of the Agent. These include

    * `alive`: which keeps track of whether the agent is alive or not 
    
    * `bump`: which tracks if the agent collides with an edge of the environment (for eg, a wall in a park)
    
    * `holding`: which is a list containing the `Things` an agent is holding, 
    
    * `performance`: which evaluates the performance metrics of the agent 
    
    * `program`: which is the agent program and maps an agent's percepts to actions in the environment. If no implementation is provided, it defaults to asking the user to provide actions for each percept.
    
* `can_grab(self, thing)`: Is used when an environment contains things that an agent can grab and carry. By default, an agent can carry nothing.

<h2 align="left"> I.3. ENVIRONMENT </h2>
Now, let us see how environments are defined. Running the next cell will display an implementation of the abstract `Environment` class.

In [59]:
psource(Environment)

`Environment` class has lot of methods! But most of them are incredibly simple, so let's see the ones we'll be using in this notebook.

* `thing_classes(self)`: Returns a static array of `Thing` sub-classes that determine what things are allowed in the environment and what aren't

* `add_thing(self, thing, location=None)`: Adds a thing to the environment at location

* `run(self, steps)`: Runs an environment with the agent in it for a given number of steps.

* `is_done(self)`: Returns true if the objective of the agent and the environment has been completed

The next two functions must be implemented by each subclasses of `Environment` for the agent to recieve percepts and execute actions 

* `percept(self, agent)`: Given an agent, this method returns a list of percepts that the agent sees at the current time

* `execute_action(self, agent, action)`: The environment reacts to an action performed by a given agent. The changes may result in agent experiencing new percepts or other elements reacting to agent input.

<h2 align="left"> I.4. Example: Blind Dog agent  </h2>



<h3 align="left"> I.4.a. Simple agent and environment  </h3>

Let's begin by using the `Agent` class to creating our first agent - a blind dog.

In [60]:
class BlindDog(Agent):
    def eat(self, thing):
        print("Dog: Ate food at {}.".format(self.location))
            
    def drink(self, thing):
        print("Dog: Drank water at {}.".format( self.location))

dog = BlindDog()

Can't find a valid program for BlindDog, falling back to default.


What we have just done is create a dog who can only feel what's in his location (since he's blind), and can eat or drink. Let's see if he's alive...

In [61]:
print(dog.alive)

True


![Cool dog](https://gifgun.files.wordpress.com/2015/07/wpid-wp-1435860392895.gif)


This is our dog. How cool is he? Well, he's hungry and needs to go search for food. For him to do this, we need to give him a program. But before that, let's create a park for our dog to play in.

<h4 align="left"> ENVIRONMENT </h4>

A park is an example of an environment because our dog can perceive and act upon it. The <b>Environment</b> class is an abstract class, so we will have to create our own subclass from it before we can use it.

In [62]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


<h4 align="left">BlindDog agent </h4>
Now that we have a <b>Park</b> Class, we re-implement our <b>BlindDog</b> to be able to move down and eat food or drink water only if it is present.


In [63]:
class BlindDog(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False


<h4 align="left"> Simple reflex agent program </h4>

A simple reflex agent program selects actions on the basis of the *current* percept, ignoring the rest of the percept history. These agents work on a **condition-action rule** (also called **situation-action rule**, **production** or **if-then rule**), which tells the agent the action to trigger when a particular situation is encountered.  

The schematic diagram below will make this more clear:

![simple_reflex_agent](images/simple_reflex_agent.jpg)

Let us now create a simple reflex agent for the environment.

The simple reflex agent program controls how the dog acts upon its environment as shown in the table below.
<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>move down</td>
   </tr>
        
</table>

In [64]:
def SimpleReflexAgentProgram():
    """This agent takes action based solely on the percept. """
    def program(percepts):
        '''Returns an action based on the dog's percepts'''
        for p in percepts:
            if isinstance(p, Food):
                return 'eat'
            elif isinstance(p, Water):
                return 'drink'
        return 'move down'
    return program

Let's now run our simulation by creating a park with some food, water, and our dog.

In [65]:
park = Park()
program = SimpleReflexAgentProgram()
simple_reflex_dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(simple_reflex_dog, 1)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

park.run(5)

BlindDog decided to move down at location: 1
BlindDog decided to move down at location: 2
BlindDog decided to move down at location: 3
BlindDog decided to move down at location: 4
BlindDog ate Food at location: 5


Notice that the dog moved from location 1 to 4, over 4 steps, and ate food at location 5 in the 5th step.

Let's continue this simulation for 5 more steps.

In [66]:
park.run(5)

BlindDog decided to move down at location: 5
BlindDog decided to move down at location: 6
BlindDog drank Water at location: 7


Perfect! Note how the simulation stopped after the dog drank the water - exhausting all the food and water ends our simulation, as we had defined before. Let's add some more water and see if our dog can reach it.

In [67]:
park.add_thing(water, 15)
park.run(10)

BlindDog decided to move down at location: 7
BlindDog decided to move down at location: 8
BlindDog decided to move down at location: 9
BlindDog decided to move down at location: 10
BlindDog decided to move down at location: 11
BlindDog decided to move down at location: 12
BlindDog decided to move down at location: 13
BlindDog decided to move down at location: 14
BlindDog drank Water at location: 15


Above, we learnt to implement an agent, its program, and an environment on which it acts. However, this was a very simple case. Let's try to add complexity to it by creating a 2-Dimensional environment!


<h3 align="left"> I.4.b. AGENTS IN A 2D ENVIRONMENT </h3>

For us to not read so many logs of what our dog did, we add a bit of graphics while making our Park 2D. To do so, we will need to make it a subclass of <b>GraphicEnvironment</b> instead of Environment. Parks implemented by subclassing <b>GraphicEnvironment</b> class adds these extra properties to it:

 - Our park is indexed in the 4th quadrant of the X-Y plane.
 - Every time we create a park subclassing <b>GraphicEnvironment</b>, we need to define the colors of all the things we plan to put into the park. The colors are defined in typical [<b>RGB digital 8-bit format</b>](https://en.wikipedia.org/wiki/RGB_color_model#Numeric_representations), common across the web.
 - Fences are added automatically to all parks so that our dog does not go outside the park's boundary - it just isn't safe for blind dogs to be outside the park by themselves! <b>GraphicEnvironment</b> provides `is_inbounds` function to check if our dog tries to leave the park.
 
First let us try to upgrade our 1-dimensional `Park` environment by just replacing its superclass by `GraphicEnvironment`. 

In [68]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

class BlindDog(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing
    
    def movedown(self):
        self.location[1] += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

Now let's test this new park with our same dog, food and water. We color our dog with a nice red and mark food and water with orange and blue respectively.

In [69]:
park = Park2D(5,20, color={'BlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)}) # park width is set to 5, and height to 20
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,1])
park.add_thing(dogfood, [0,5])
park.add_thing(water, [0,7])
morewater = Water()
park.add_thing(morewater, [0,15])
print("BlindDog starts at (1,1) facing downwards, lets see if he can find any food!")
park.run(20)

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


Adding some graphics was a good idea! We immediately see that the code works, but our blind dog doesn't make any use of the 2 dimensional space available to him. Let's make our dog more energetic so that he turns and moves forward, instead of always moving down. In doing so, we'll also need to make some changes to our environment to be able to handle this extra motion.

<h4 align="left">  EnergeticBlindDog Agent </h4>

Let's make our dog turn or move forwards at random - except when he's at the edge of our park - in which case we make him change his direction explicitly by turning to avoid trying to leave the park. However, our dog is blind so he wouldn't know which way to turn - he'd just have to try arbitrarily.

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>
       <table>
           <tr>
               <td><b>Remember being at Edge : </b></td>
               <td>At Edge</td>
               <td>Not at Edge</td>
           </tr>
           <tr>
               <td><b>Action : </b></td>
               <td>Turn Left / Turn Right <br> ( 50% - 50% chance )</td>
               <td>Turn Left / Turn Right / Move Forward <br> ( 25% - 25% - 50% chance )</td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

In [70]:
from random import choice

class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
        


<h4 align="left">  Simple reflex agent program </h4>
Implement a simple reflex agent program

In [71]:

def program(percepts):
    '''Returns an action based on it's percepts'''
        
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
        if isinstance(p,Bump): # then check if you are at an edge and have to turn
            turn = False
            move = choice((1,2))
        else:
            move = choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if move == 1:
        return 'turnright'
    elif move == 2:
        return 'turnleft'
    else:
        return 'moveforward'
    

<h4 align="left">ENVIRONMENT - Park2D </h4>

We also need to modify our park accordingly, in order to be able to handle all the new actions our dog wishes to execute. Additionally, we'll need to prevent our dog from moving to locations beyond our park boundary - it just isn't safe for blind dogs to be outside the park by themselves.

In [72]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = agent.location # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


Now that our park is ready for the 2D motion of our energetic dog, lets test it!

In [73]:
park = Park2D(5,5, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])
morewater = Water()
morefood = Food()
park.add_thing(morewater, [2,4])
park.add_thing(morefood, [4,3])
print("dog started at [0,0], facing down. Let's see if he found any food or water!")
park.run(20)

,,,,
,,,,
,,,,
,,,,
,,,,


<h2 align="left"> <span style="color:red"> I.5. Exercise: Vacuum-cleaner Agent </span> </h2>

<span style="color:darkblue"> **(Your turn)** </span>
 Consider a vacuum cleaner that operates in a two-square environment, as illustrated in the Figure below.
![Vacuum cleaner](images/fig2-2.png)

<h3 align="left"> I.5.a. Questions </h3>

    1. What are the characteristics of the environment?
        In this case, the environment is an array of 2 elements corresponding to the 2 squares.
    2. Specify the information about percepts.
        The precepts are the state of the square (dirty or clean).
    3. Identify the set of actions.
        The set of actions is {suck, move left, move right}
    4. Enumerate the set of rules for the reflex agent
        The set of rules is:
            - if the square is dirty, suck
            - if the square is clean, move right


<h3 align="left"> I.5.b. Implementation </h3>

    1. Implement the environment with its states.
    2. Implement a simple reflex agent.
    3. Simulate the agent's behavior in the environment to demonstrate its cleaning action.
        - Instantiate the environment and the agent.
        - Place the agent in the environment with a random state (dirty or clean for each square).
        - Let the agent operate until both squares are clean.
     4. Implement a Model-based reflex agent and simulate the agent's behavior in the environment.

<h3 align="left"> <span style="color:darkblue"> <b> I.5.c. Bonus: </b> </span>  </h3>

Add complexity to this exercise by creating a 2-dimensional vacuum cleaner environment! and simulating a simple reflex agent.
    
    1. Implement a simple reflex agent and simulate the agent's behavior in the 2-D environment.
    2. Implement a Model-based reflex agent and simulate the agent's behavior in the 2-D environment.
   
**Remark:** a graphical visualization is not necessarily required.

In [74]:
class Dirt(Thing):
    pass

class Vaccum(Agent):
    location = 0
    
    def moveright(self):
        self.location += 1
    def moveleft(self):
        self.location -= 1
        
    def suck(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Dirt):
            return True
        return False
    

In [75]:
class Room(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move right":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.moveright()
        elif action == "move left":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.moveleft()
        elif action == "NoOp":
            print('The room is clean')
            
        elif action == "suck":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.suck(items[0]): #Have the dog eat the first item
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
       

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_dirt = not any(isinstance(thing, Dirt) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_dirt


In [76]:
def SimpleReflexAgentProgramVaccum():
    """This agent takes action based solely on the percept. """
    def program(percepts):
        '''Returns an action based on the dog's percepts'''
        for p in percepts:
            if isinstance(p, Dirt):
                return "suck"
        if percepts[0].location == 0:
            return "move right"
        elif percepts[0].location == 1:
            return "move left"
    return program

def ModelBasedReflexAgentProgramVaccum(loc_A=None, loc_B=None):
    """This agent takes action based on the percept and state. """
    model = {loc_A: None, loc_B: None}
    def program(percept):
        """Same as ReflexVacuumAgent, except if everything is clean, do NoOp."""
        location = percept[0]
        status = percept[1]
        model[location] = status  # Update the model here
        if model[loc_A] == model[loc_B] == 'Clean':
            return 'NoOp'
        elif status == 'Dirt':
            return 'suck'
        elif location == loc_A:
            return 'move right'
        elif location == loc_B:
            return 'move left'
    return program

In [77]:
room = Room()
program = SimpleReflexAgentProgramVaccum()
simple_reflex_vaccum = Vaccum(program)
dirt1 = Dirt()
dirt2 = Dirt()
room.add_thing(simple_reflex_vaccum, 0)
room.add_thing(dirt1, 0)
room.add_thing(dirt2, 1)
room.run(5)

Vaccum cleaned Dirt at location: 0
Vaccum decided to move right at location: 0
Vaccum cleaned Dirt at location: 1


In [79]:
newRoom = Room()
modelprogram = ModelBasedReflexAgentProgramVaccum()
model_reflex_vaccum = Vaccum(modelprogram)
dirt3 = Dirt()
dirt4 = Dirt()
room.add_thing(model_reflex_vaccum, 0)
room.add_thing(dirt3, 0)
room.add_thing(dirt4, 1)
room.run(5)


Vaccum cleaned Dirt at location: 1
Vaccum decided to move left at location: 1
Vaccum cleaned Dirt at location: 0


In [80]:
class Room2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = agent.location # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "suck":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.suck(items[0]):
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_dirt = not any(isinstance(thing, Dirt)  for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_dirt

In [81]:
class EnergeticVaccum(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def suck(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Dirt):
            return True
        return False
    
    

In [82]:
def vaccumprogram(percepts):
    '''Returns an action based on it's percepts'''
        
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Dirt):
            return 'suck'
        if isinstance(p,Bump): # then check if you are at an edge and have to turn
            turn = False
            move = choice((1,2))
        else:
            move = choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if move == 1:
        return 'turnright'
    elif move == 2:
        return 'turnleft'
    else:
        return 'moveforward'


In [83]:
room2d = Room2D(5,5, color={'EnergeticVaccum': (200,0,0), 'Dirt': (0, 200, 200)})
vaccum = EnergeticVaccum(vaccumprogram)
dirt = Dirt()
dirt0 = Dirt()
room2d.add_thing(vaccum, [0,0])
room2d.add_thing(dirt, [1,2])
room2d.add_thing(dirt0, [0,1])
room2d.run(20)


,,,,
,,,,
,,,,
,,,,
,,,,
